# Build the Neural Network

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

## Get Device for Training

In [2]:
device=(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Define the Class

In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )

  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1,28,28,device=device)
logits=model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


## Model Layers

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [10]:
layer1 = nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 1.0508, -0.1001, -0.1934,  0.5044,  0.2829,  0.0400,  0.5549,  0.3996,
          0.3313,  0.0264, -0.0231,  0.1750,  0.2391, -0.5302, -0.3430,  0.0964,
         -0.2310,  0.1447,  0.2736, -0.3575],
        [ 1.0549, -0.1324, -0.1703,  0.2565,  0.7533, -0.0139,  0.2773,  0.0340,
          0.3770,  0.2174,  0.1873, -0.0497,  0.4042, -0.4166, -0.2902, -0.0410,
         -0.3802,  0.0597,  0.6245, -0.2852],
        [ 1.1293, -0.0788, -0.4091,  0.3393,  0.3992,  0.1046,  0.4166, -0.1021,
          0.3724, -0.2542,  0.1152,  0.3190,  0.4427, -0.4392,  0.0242, -0.2638,
         -0.6615, -0.2519,  0.3880, -0.0514]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[1.0508, 0.0000, 0.0000, 0.5044, 0.2829, 0.0400, 0.5549, 0.3996, 0.3313,
         0.0264, 0.0000, 0.1750, 0.2391, 0.0000, 0.0000, 0.0964, 0.0000, 0.1447,
         0.2736, 0.0000],
        [1.0549, 0.0000, 0.0000, 0.2565, 0.7533, 0.0000, 0.2773, 0.0340, 0.3770,
         0.2174, 0.1873, 0.0000, 0.4042, 0.0000, 0.00

### nn.Sequential

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

In [16]:
print(f"Model structure: {model}\n\n")
for name,param in model.named_parameters():
  print(f"Layer:{name} | Size:{param.size()} | Values:{param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer:linear_relu_stack.0.weight | Size:torch.Size([512, 784]) | Values:tensor([[ 0.0024,  0.0265,  0.0350,  ...,  0.0243,  0.0332, -0.0356],
        [-0.0069, -0.0150,  0.0191,  ...,  0.0135, -0.0357,  0.0238]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.0.bias | Size:torch.Size([512]) | Values:tensor([-0.0073,  0.0229], device='cuda:0', grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.2.weight | Size:torch.Size([512, 512]) | Values:tensor([[ 0.0277,  0.0173,  0.0331,  ..., -0.0080, -0.0075,  0.0195],
        [-0.0019,  0.0303, -0.0399,  ..., -0.0151, -0.0365, -0.0431]],
       device='cuda:0', grad_fn=<SliceBackward0>)